In [1]:
import pandas as pd
import numpy as np
import datetime

#%matplotlib inline

# Cargo CSV

In [2]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [3]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

In [4]:
vistas = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index().drop('index',axis=1)

In [5]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
postulaciones = postulaciones.reset_index().drop('index',axis=1)

In [6]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

# Pre-Procesamiento de datos

### Limpio el DF *postulantes_genero_y_edad*

 Elimino los duplicados dejando el ultimo suponiendo que es la ultima actualizacion

In [7]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()

In [8]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')

Reemplazo los valores invalidos de la columna sexo por NO_DECLARA

In [9]:
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

Calculo la edad de los postulantes y relleno las que quedan en Nan con la media

In [10]:
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [11]:
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int32')

In [12]:
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop('index',axis=1)

### Limpio el DF *vistas*

Elimino duplicados y me quedo con el ultimo suponiendo que es la ultima actualizacion

In [13]:
vistas.isnull().sum()

idAviso         0
timestamp       0
idpostulante    0
dtype: int64

In [14]:
vistas = vistas.drop_duplicates(keep='last')

In [15]:
vistas = vistas.drop_duplicates(subset=['idAviso','idpostulante'],keep='last')
vistas = vistas.rename(columns={'idAviso': 'idaviso'})
vistas = vistas.reset_index().drop('index',axis=1)

### Limpio el DF *postulaciones*

Elimino duplicados y me quedo con el ultimo

In [16]:
postulaciones = postulaciones.drop_duplicates(keep='last')

In [17]:
postulaciones = postulaciones.drop_duplicates(subset=['idaviso','idpostulante'],keep='last')

In [18]:
postulaciones.shape

(6603752, 3)

In [19]:
postulaciones = postulaciones.reset_index().drop('index',axis=1)

### Limpio el DF avisos_detalle

Elimino las columnas que no necesito y elimino duplicados

In [20]:
avisos_detalle = avisos_detalle.drop(['ciudad','mapacalle','nombre_zona','descripcion','titulo','tipo_de_trabajo','nivel_laboral','denominacion_empresa','idpais'],axis=1)

avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

# Gente postulada y no postulada

La gente no postulada es aquella que no aparece en el df postulaciones

In [21]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [22]:
gente_no_postulada['sexo'].value_counts()

FEM           71992
MASC          67819
NO_DECLARA    15713
Name: sexo, dtype: int64

In [23]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [24]:
gente_postulada['sexo'].value_counts()

FEM           179213
MASC          160034
NO_DECLARA      9636
Name: sexo, dtype: int64

In [25]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [26]:
gente_postulada.isnull().sum()

idpostulante           0
fechanacimiento    10074
sexo                   0
edad                   0
dtype: int64

# Funciones de utilidad

In [27]:
# fijarse de balancear el sample con 50% de postulados y no postulados

# Generador de sample
# el id tiene que estar en la columna 0
# Recibe los DFs de idpostulante e idaviso
def generador_de_sample_df(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = idpostulante.drop(columns=idpostulante.columns[1:])
    idaviso = idaviso.drop(columns=idaviso.columns[1:])
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [28]:
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop('tem',axis=1)
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [29]:
def borrar_tuplas_test_del_sample(sample,test_final):
    df_return = sample.merge(test_final,how='left',on=['idaviso','idpostulante'],indicator=True)
    df_return = df_return[df_return['_merge'] == 'left_only']
    df_return = df_return.drop('_merge',axis=1)
    return df_return

In [30]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Funciones feature

In [31]:
def feature_postulante_se_postulo_al_aviso(tupla_idaviso_idpostulante,postulaciones):
    df_return = tupla_idaviso_idpostulante.merge(postulaciones,on=['idaviso','idpostulante'],how='left')
    return df_return

In [32]:
def feature_genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    return df_return

In [33]:
def feature_visto(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(vistas,on=['idaviso','idpostulante'],how='left')
    return df_return

# Tupla postulada

In [34]:
tupla_postulada = postulaciones.drop('fechapostulacion',axis=1)
tupla_postulada['y'] = 1

In [35]:
tupla_postulada.head()

,idaviso,idpostulante,y
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [36]:
tupla_postulada.shape

(6603752, 3)

# Tupla no postulada

Genero el sample con todos los avisos y la gente que no se postulo nunca

In [37]:
tupla_no_postulada = generador_de_sample_serie(avisos_detalle['idaviso'],gente_no_postulada['idpostulante'],100,100)
tupla_no_postulada['y'] = 0

In [38]:
tupla_no_postulada.head()

,idaviso,idpostulante,y
0,1112390041,ow240Br,0
1,1112390041,ZDxOzz,0
2,1112390041,xkvqarz,0
3,1112390041,JBdWZAr,0
4,1112390041,X95z0Nd,0


# Features en Tupla Final

Creo el sample con 50% postulados y 50% no postulados

In [39]:
sample = tupla_no_postulada.append(tupla_postulada.sample(n=1000000,random_state=0))

In [40]:
sample['y'].value_counts()

1    1000000
0    1000000
Name: y, dtype: int64

### Feature Genero

In [41]:
sample = feature_genero_edad(sample,gente_postulada.append(gente_no_postulada))
sample  = sample.drop('fechanacimiento',axis=1)
sample  = oneHotEncoding(sample,'sexo')

### Feature visto

Agrego la columna 'visto' al csv de vistas que indica con un 1 si el aviso fue visto por cierto postulante

In [42]:
vistas['visto'] = 1
vistas = vistas.drop('timestamp',axis=1)

 Se hace un merge de las tuplas con el csv de vistas, si la columna 'visto' queda en Nan significa que ese aviso no fue visto por ese postulante

In [43]:
sample=feature_visto(sample)

Los que quedaron en Nan se reemplazan por 0

sample.isnull().sum()

In [44]:
sample = sample.fillna(0)

In [45]:
sample.visto.value_counts()

0.0    1730996
1.0     269004
Name: visto, dtype: int64

In [46]:
sample = sample.drop(['idaviso','idpostulante'],axis=1)

In [47]:
sample.head()

,y,edad,FEM,MASC,NO_DECLARA,visto
0,0,59,1,0,0,0.0
1,0,42,0,1,0,0.0
2,0,20,1,0,0,0.0
3,0,21,0,1,0,0.0
4,0,27,0,1,0,0.0


# Features en Test Final

### Feature Genero

In [48]:
test_final = feature_genero_edad(test_final,gente_postulada.append(gente_no_postulada))

In [49]:
# Salvo datos nulos
test_final.loc[test_final['sexo'].isnull(),'sexo'] = 'NO_DECLARA'
test_final.loc[test_final['edad'].isnull(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())

In [50]:
test_final['edad'] = test_final['edad'].astype('int32')

In [51]:
test_final.isnull().sum()

idaviso               0
idpostulante          0
fechanacimiento    2838
sexo                  0
edad                  0
dtype: int64

In [52]:
test_final = oneHotEncoding(test_final,'sexo')

In [53]:
test_final.head()

,idaviso,idpostulante,fechanacimiento,edad,FEM,MASC,NO_DECLARA
0,739260,6M9ZQR,1976-03-29,42,1,0,0
1,739260,6v1xdL,1987-06-27,31,0,1,0
2,739260,ezRKm9,1982-02-17,36,1,0,0
3,758580,1Q35ej,1949-12-24,69,0,1,0
4,758580,EAN4J6,1986-05-27,32,1,0,0


### Feature Visto

In [54]:
#Lo mismo que hice con el sample antes lo hago con el test_final
test_final = test_final.merge(vistas,how='left',on=['idaviso','idpostulante'])

In [55]:
test_final.isnull().sum()

idaviso                0
idpostulante           0
fechanacimiento     2838
edad                   0
FEM                    0
MASC                   0
NO_DECLARA             0
visto              65839
dtype: int64

In [56]:
test_final = test_final.fillna(0)

In [57]:
test_final.visto.value_counts()

0.0    65839
1.0    34161
Name: visto, dtype: int64

In [58]:
test_final = test_final.drop(['idaviso','idpostulante','fechanacimiento'],axis=1)

In [59]:
test_final.head()

,edad,FEM,MASC,NO_DECLARA,visto
0,42,1,0,0,0.0
1,31,0,1,0,0.0
2,36,1,0,0,0.0
3,69,0,1,0,0.0
4,32,1,0,0,0.0


# Prueba Local

In [60]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(sample.drop(columns='y'),sample['y'],random_state=0,test_size=0.01)

In [61]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [62]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [63]:
ModeloRFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [64]:
result = ModeloRFC.predict(X_test)

In [65]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,result)

0.6443041247670347

In [66]:
#-----------------------------------------------------------------
# Modelo GradientBoostingClassifier
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingClassifier

In [67]:
modeloGBC=GradientBoostingClassifier(n_estimators=100)

In [68]:
modeloGBC.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [69]:
result = modeloGBC.predict(X_test)

In [70]:
roc_auc_score(y_test,result)

0.6440990000409609

# Prueba con Test Final

In [ ]:
train_label = []
train_data = []

train_label = sample.iloc[:, 0]
train_data = sample.iloc[:, sample.columns != 'y']

In [ ]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [ ]:
ModeloRFC = RandomForestClassifier(n_estimators=100,random_state=0, n_jobs=-1)

In [ ]:
ModeloRFC.fit(train_data, train_label)

In [ ]:
result = ModeloRFC.predict(test_final)

In [ ]:
result.size

In [ ]:
import csv

In [ ]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'sepostulo'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [ ]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/RFC_featurevisto_final.csv")

In [ ]:
prueba = pd.read_csv('../predicciones/RFC_featurevisto_final.csv')

In [ ]:
prueba.sepostulo.value_counts()

In [ ]:
del ModeloRFC

In [ ]:
#-----------------------------------------------------------------
# Modelo GradientBoostingClassifier
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
modeloGBC=GradientBoostingClassifier(n_estimators=100)

In [ ]:
modeloGBC.fit(train_data, train_label)

In [ ]:
result = modeloGBC.predict(test_final)

In [ ]:
result.size

In [ ]:
write_submission("../data/test_final_100k.csv", result, "../predicciones/GBC_featurevisto_final.csv")

In [ ]:
prueba = pd.read_csv('../predicciones/GBC_featurevisto_final.csv')

In [ ]:
prueba.sepostulo.value_counts()